<a href="https://colab.research.google.com/github/DanielMesaB/Integracion-y-prospectiva-de-datos-2025-01/blob/main/5_Integraci%C3%B3nDinamica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
0. Se procede con la carga e instalación de las librerias de trabajo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Se procede con la carga de los archivos de trabajo

In [ ]:
nxl='/content/drive/MyDrive/INTEGRACIÓN DE DATOS 2025-1/BASE DATOS NBA.xlsx'
XDB= pd.read_excel(nxl,sheet_name=0) # Base de datos observada
XDB.head(10)

XDB=XDB.iloc[:,1:6] # Para seleccionar solo las variables que me interesan ( Salario, edad, altura,peso y juegos jugados)
XDB.dropna()
XDB.head(10)


2. Se procede con el proceso de clusterización estatica (K-means)

In [ ]:
np.random.seed(42) # Grupo de npumeros aleatorios para que den los mismos datos

#1. Se deben crear las semillas iniciales
XDBM=np.max(XDB,axis=0) # Se crean los maximos de las variables, # el axis, es para que los haga por columna
XDBm=np.min(XDB,axis=0) # Se crean los minimos de las variables, # axis=0 es columnas


XC=np.zeros((5,5)) # Se crean 5 montonsitos, el primer 5 son las variables que necesitamos, y el segundo son las variables que teniamos

for i in range(5): # Nos indica el número de montoncitos
  for j in range(5): # Cada montoncito va a tener 5 variables
    XC[i,j]=np.random.uniform(XDBm[j],XDBM[j])

print (XC.astype(int)) # Me quita los decimales, me los muestra enteros

XDB=np.array(XDB) # Se cambia el dataframe a un array
XCd= np.zeros((5,5)) # Los clusters despues)
ND= np.zeros((5,1))
#2. Se procede con la clasificación de los datos por montoncitos

for k in range (len(XDB)):
  d=np.sum(np.abs(XC[:,:]-XDB[k,:]),axis=1) # Distancia de un registro a un cluster, axis = 1 es para filas
  fila=np.where(d==np.min(d))[0][0] # Número de cluster al que pertenece un dato # Otra alternativa es fila=np.argmin(d)

  XCd[fila,]=XCd[fila,]+XDB[k,:] # Voy sumando la info de las variables a los clusters
  ND[fila,]=ND[fila,]+1   # Cuento el número de datos que quedaron por cluster

print("El número de datos por cluster es\n", ND)


XCd=XCd/ND # Ahora dividimos todas las variables por el número de datos del cluster
dfxco=pd.DataFrame((XCd.astype(float)))
dfxco.columns=['Salary','Age','Height','Weight','Games_played']
dfxco.head(5)
# Se procede a mostrar la correlación entre las variables
dfcor=pd.DataFrame((XDB)).corr()
print("La correlación entre las variables es:\n", dfcor)


[[ 9433206       39       82      248      238]
 [ 3993922       23       84      248     1038]
 [  623762       39       83      192      275]
 [ 4676117       28       78      224      434]
 [15339579       25       74      214      673]]
El número de datos por cluster es
 [[50.]
 [42.]
 [62.]
 [45.]
 [50.]]
La correlación entre las variables es:
           0         1         2         3         4
0  1.000000 -0.243711  0.069705  0.098346  0.066004
1 -0.243711  1.000000  0.005723  0.015531  0.787159
2  0.069705  0.005723  1.000000  0.857509 -0.022546
3  0.098346  0.015531  0.857509  1.000000  0.011834
4  0.066004  0.787159 -0.022546  0.011834  1.000000


##**Integración Dinamica**


1. Se procede con la carga de los datos externos

In [ ]:
#1. Se carga la base de datos
nxle='/content/drive/MyDrive/INTEGRACIÓN DE DATOS 2025-1/BASE DATOS NBA.xlsx'
XDBe=pd.read_excel(nxle,sheet_name=1)
XDBe=np.array(XDBe.iloc[:,1:6].dropna())
print("Los datos externos son:\n",XDBe)

#2. Mantenemos el historial de los clusters
XCe=np.copy(XCd) #Cluster estáticos o iniciales
NDe=np.zeros((5,1)) # Número de datos que se integraron por cluster
nf=np.zeros((len(XDBe),1)) #Para saber la fila de integración o cluster de recepción

for k in range (len(XDBe)):
  de=np.sqrt(np.sum((XCe[:,:]-XDBe[k,:])**2,axis=1)) # Distancia de un dato externo a los clusters internos
  fila=np.argmin(de) # Número de cluster al que pertenece un dato
  nf[k,]=fila
  XCe[fila,]=(XCe[fila,]+XDBe[k,:])/2 # Voy sumando la información de las variables a los clusters

  NDe[fila,]=NDe[fila,]+1  # Cuento el número de datos que quedaron por el cluster

print("El número de datos externos integrados por cluster es:\n",NDe)
print("La estrcutura de los cluster antes es:\n",pd.DataFrame(XCd))
print("La estructura de los cluster después es:\n",pd.DataFrame(XCe))

#Los cambios porcentuales son:
print("Los cambios porcentuales fueron:\n", (XCd-XCe)/XCd)

# El informe de integración es el sguiente
dfinf=pd.DataFrame(np.column_stack((nf,XDBe)))
dfinf.columns=['Cluster','Salary','Age','Height','Weight','Games_played']
dfinf.head(10)


Los datos externos son:
 [[  981348       25       79      240      219]
 [ 3873398       23       80      225      307]
 [ 3533333       25       78      175      254]
 [ 5694674       22       77      207      247]
 [ 6796117       25       78      235      311]
 [  947276       24       76      195       84]
 [ 7070730       23       82      253      260]
 [ 3272091       22       83      279      304]
 [ 2008748       26       83      265      170]
 [ 2288205       23       79      205      251]
 [14260870       26       79      230      321]
 [ 2894059       23       81      215      279]
 [ 2943221       25       83      229      257]
 [  981348       25       76      215      141]
 [   55722       27       77      220      103]
 [ 2489530       24       81      252      178]
 [ 6331404       22       79      232      202]
 [ 3034356       23       77      185      214]
 [ 3075880       24       85      245      225]
 [ 4236287       25       75      195      321]
 [ 2144772     

,Cluster,Salary,Age,Height,Weight,Games_played
0,2.0,981348.0,25.0,79.0,240.0,219.0
1,1.0,3873398.0,23.0,80.0,225.0,307.0
2,1.0,3533333.0,25.0,78.0,175.0,254.0
3,3.0,5694674.0,22.0,77.0,207.0,247.0
4,3.0,6796117.0,25.0,78.0,235.0,311.0
5,2.0,947276.0,24.0,76.0,195.0,84.0
6,3.0,7070730.0,23.0,82.0,253.0,260.0
7,1.0,3272091.0,22.0,83.0,279.0,304.0
8,2.0,2008748.0,26.0,83.0,265.0,170.0
9,2.0,2288205.0,23.0,79.0,205.0,251.0


##**Analisis de resultados**##
En la integración de los datos, se pueede observar que el cluster que tuvo la menor integración  o una integración nula, fue el csluter 0, es por esto que los valores asociados a los cluster tuvieron  una variación  nula tal como se puede observar, en la matriz de variaciones porcentuales.
Se destaca el cluster 2, el cual tuvo la mayor cantidad de datps de integración, sin embargo, esta integracióon tuvo variaciones por encima del 80% en promedio con respecto a la variable dames_played. Es por esto que se requiere establecer controles de integración con el fin de que los datos no tengan variaciones tan fuertes que pueden afectar la estrcutura de los clusters originales o asociados con la base de datos observada.
